In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

# ======================
# CONFIG
# ======================
# Because the notebook is in "notebooks/", dataset is one folder above
data_dir = "../dataset"   
batch_size = 8
num_epochs = 10
learning_rate = 0.001
num_classes = 2  # helmet / no_helmet
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Check that dataset path is correct
print("Using dataset path:", os.path.abspath(data_dir))

# ======================
# TRANSFORMS
# ======================
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),  # data augmentation
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], 
                         [0.229, 0.224, 0.225])  # ImageNet normalization
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], 
                         [0.229, 0.224, 0.225])
])

# ======================
# DATA LOADERS
# ======================
train_data = datasets.ImageFolder(root=f"{data_dir}/train", transform=train_transforms)
val_data = datasets.ImageFolder(root=f"{data_dir}/val", transform=val_transforms)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

print("Classes:", train_data.classes)  # should print ['helmet', 'no_helmet']


# ======================
# MODEL (Transfer Learning)
# ======================
model = models.resnet18(pretrained=True)   # load pretrained ResNet18
model.fc = nn.Linear(model.fc.in_features, num_classes)  # adjust final layer
model = model.to(device)

# ======================
# LOSS & OPTIMIZER
# ======================
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# ======================
# TRAINING LOOP
# ======================
for epoch in range(num_epochs):
    # --- Training ---
    model.train()
    running_loss, running_corrects = 0.0, 0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    
    epoch_loss = running_loss / len(train_data)
    epoch_acc = running_corrects.double() / len(train_data)
    
    # --- Validation ---
    model.eval()
    val_loss, val_corrects = 0.0, 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            _, preds = torch.max(outputs, 1)
            val_loss += loss.item() * inputs.size(0)
            val_corrects += torch.sum(preds == labels.data)
    
    val_loss /= len(val_data)
    val_acc = val_corrects.double() / len(val_data)
    
    print(f"Epoch {epoch+1}/{num_epochs} "
          f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} | "
          f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

# ======================
# SAVE MODEL
# ======================
torch.save(model.state_dict(), "helmet_classifier.pth")
print("Model saved as helmet_classifier.pth")


Using dataset path: c:\Users\zulfa\INF3001_Project\dataset
Classes: ['helmet', 'no_helmet']


c:\Users\zulfa\INF3001_Project\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\zulfa\INF3001_Project\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


KeyboardInterrupt: 